In [2]:
FLATTENED_DATA_FOLDER_PATH = '../data/flattened'
from __future__ import print_function
import tensorflow as tf
import numpy as np
import os


C:\Anaconda_3_5\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [32]:
def one_hot(arr, num_classes=n_classes):
    return np.eye(num_classes)[arr.astype(int)]

In [30]:

# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 10000
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_input = 40 # MNIST data input (img shape: 28*28)
n_classes = 138 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
   'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()


In [34]:
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        print("********************Starting Epoch: ",epoch)
        avg_cost = 0.
        d = Data(batch_size=batch_size)
        # Loop over all batches
        for batch_x, batch_y in d.load_train_batchwise():
            # Run optimization op (backprop) and cost op (to get loss value)
            batch_y_one_hot = one_hot(batch_y, n_classes)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y_one_hot})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    devX, devY = d.load_dev() 
    print("Accuracy:", accuracy.eval({X: devX, Y: one_hot(devY, n_classes)}))

********************Starting Epoch:  0
***************Current Batch:


ResourceExhaustedError: OOM when allocating tensor with shape[1000000,138] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: MatMul_8 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Add_5, Variable_14/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: Mean_2/_15 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_245_Mean_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'MatMul_8', defined at:
  File "C:\Anacond

In [17]:
class Data:
    def __init__(self, data_folder_path=FLATTENED_DATA_FOLDER_PATH, batch_size=1000000, n_context=0):
        self.batch_size = batch_size
        self.n_context = n_context
        self.path = data_folder_path
        self.curr_batch_idx = 0
    def load_train_batchwise(self):
        TRAIN = 'train'
        self.trainLookup = np.load(os.path.join(FLATTENED_DATA_FOLDER_PATH, TRAIN + 'FLATTENED_Lookup.npy')).astype(int)
        self.trainXmem_map = np.load(os.path.join(FLATTENED_DATA_FOLDER_PATH, TRAIN + 'FLATTENED_X.npy'),mmap_mode="r")
        self.trainYmem_map = np.load(os.path.join(FLATTENED_DATA_FOLDER_PATH, TRAIN + 'FLATTENED_Y.npy'),mmap_mode="r")
        
        self.total_num_batch = int(self.trainLookup.shape[0]/self.batch_size)+1
        Lookup_indexes = np.array(range(self.trainLookup.shape[0]))
        np.random.shuffle(Lookup_indexes)
        
        for curr_batch in range(self.total_num_batch):
            print("***************Current Batch:", )
            start_idx = curr_batch * self.batch_size
            end_idx = start_idx + self.batch_size
            batch_indx = self.trainLookup[Lookup_indexes[start_idx: end_idx]]
            X, Y = self.trainXmem_map[batch_indx], self.trainYmem_map[Lookup_indexes[start_idx: end_idx]]
            yield np.array(X), np.array(Y)
    def load_dev(self):
        self.devLookup = np.load(os.path.join(FLATTENED_DATA_FOLDER_PATH, 'dev' + 'FLATTENED_Lookup.npy')).astype(int)
        self.devXmem_map = np.load(os.path.join(FLATTENED_DATA_FOLDER_PATH, 'dev' + 'FLATTENED_X.npy'))
        self.devYmem_map = np.load(os.path.join(FLATTENED_DATA_FOLDER_PATH, 'dev' + 'FLATTENED_Y.npy'))
        
        return self.devXmem_map[self.devLookup], self.devYmem_map[:]

In [18]:
d=Data()

In [19]:
x,y = d.load_dev()

In [20]:
x.shape, y.shape

((669294, 40), (669294,))

In [27]:
y = d.load_dev()[1]
np.eye(n_classes)[y.astype(int)]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])